# VDEH Missing Values Analyse

**Fokus:** Identifikation lückenbehafteter Spalten und Vollständigkeits-Statistiken

## 🎯 Ziel
- Schneller Überblick über fehlende Werte pro Spalte
- Vollständigkeits-Score pro Record
- Visualisierung der Lücken-Verteilung

## 📚 Input/Output
- **Input**: `data/vdeh/processed/03_language_detected_data.parquet`
- **Output**: `data/vdeh/processed/04_quality_analyzed_data.parquet` (mit Quality Scores)

In [1]:
# 🛠️ SETUP UND DATEN LADEN
import sys
from pathlib import Path

✅ Konfiguration geladen: /media/sz/Data/Bibo/analysis/config.yaml
📁 Projektroot: /media/sz/Data/Bibo/analysis
✅ Konfiguration geladen
📊 Matplotlib konfiguriert: [12, 8]


In [2]:
# 📂 DATEN AUS VORHERIGER STUFE LADEN
processed_dir = config.project_root / config.get('paths.data.vdeh.processed')
input_path = processed_dir / '03_language_detected_data.parquet'
metadata_path = processed_dir / '03_metadata.json'

if not input_path.exists():
    raise FileNotFoundError(f"Input-Datei nicht gefunden: {input_path}\n"
                          "Bitte führen Sie zuerst 03_vdeh_language_detection.ipynb aus.")

# Daten laden
df_vdeh = pd.read_parquet(input_path)

# Vorherige Metadaten laden
with open(metadata_path, 'r') as f:
    prev_metadata = json.load(f)

print(f"📂 Daten geladen aus: {input_path}")
print(f"📊 Records: {len(df_vdeh):,}")
print(f"📋 Spalten: {list(df_vdeh.columns)}")
print(f"📅 Vorherige Verarbeitung: {prev_metadata['processing_date']}")
print(f"🌍 Sprach-Analyse: {prev_metadata['language_analysis']['total_titles_analyzed']:,} Titel analysiert")

📂 Daten geladen aus: /media/sz/Data/Bibo/analysis/data/vdeh/processed/03_language_detected_data.parquet
📊 Records: 58,760
📋 Spalten: ['id', 'title', 'authors', 'authors_affiliation', 'year', 'publisher', 'isbn', 'issn', 'authors_str', 'num_authors', 'authors_affiliation_str', 'num_authors_affiliation', 'isbn_valid', 'isbn_status', 'issn_valid', 'issn_status', 'lang_code', 'lang_confidence', 'lang_name']
📅 Vorherige Verarbeitung: 2025-11-05T07:37:03.330582
🌍 Sprach-Analyse: 40,544 Titel analysiert


In [11]:
# 🔍 MISSING VALUES ANALYSE
print("🔍 === MISSING VALUES ANALYSE ===\n")

# Relevante Spalten für Analyse
analysis_cols = ['title', 'authors_str', 'year', 'publisher', 'isbn', 'issn']

# Nur vorhandene Spalten verwenden
available_cols = [col for col in analysis_cols if col in df_vdeh.columns]

print(f"📊 Fehlende Werte pro Spalte (von {len(df_vdeh):,} Records):\n")

# Erstelle Übersicht sortiert nach Missing Rate
missing_stats = []
for col in available_cols:
    # Spezialbehandlung für authors_str: auch leere Strings als fehlend betrachten
    if col == 'authors_str':
        missing_count = (df_vdeh[col].isna() | (df_vdeh[col] == '')).sum()
    else:
        missing_count = df_vdeh[col].isna().sum()
    
    missing_pct = (missing_count / len(df_vdeh)) * 100
    present_count = len(df_vdeh) - missing_count
    missing_stats.append({
        'Spalte': col,
        'Vorhanden': present_count,
        'Fehlend': missing_count,
        'Fehlend %': missing_pct
    })

# Sortiere nach Fehlend %
missing_df = pd.DataFrame(missing_stats).sort_values('Fehlend %', ascending=False)

# Ausgabe als formatierte Tabelle
for _, row in missing_df.iterrows():
    bar_length = int(row['Fehlend %'] / 2)  # Balken bis 50 Zeichen
    bar = '█' * bar_length
    status = '🔴' if row['Fehlend %'] > 50 else '🟡' if row['Fehlend %'] > 20 else '🟢'
    print(f"{status} {row['Spalte']:18} {row['Vorhanden']:7,} ✓  {row['Fehlend']:7,} ✗  {row['Fehlend %']:5.1f}% {bar}")

# Berechne Vollständigkeits-Score pro Record
# Erstelle Missing Matrix mit Spezialbehandlung für authors_str
missing_matrix = df_vdeh[available_cols].isnull()

# Für authors_str: leere Strings auch als fehlend markieren
if 'authors_str' in available_cols:
    missing_matrix['authors_str'] = df_vdeh['authors_str'].isna() | (df_vdeh['authors_str'] == '')

completeness_scores = (1 - missing_matrix.mean(axis=1)) * 100

🔍 === MISSING VALUES ANALYSE ===

📊 Fehlende Werte pro Spalte (von 58,760 Records):

🔴 issn                   721 ✓   58,039 ✗   98.8% █████████████████████████████████████████████████
🔴 isbn                11,415 ✓   47,345 ✗   80.6% ████████████████████████████████████████
🔴 authors_str         17,011 ✓   41,749 ✗   71.1% ███████████████████████████████████
🔴 publisher           23,553 ✓   35,207 ✗   59.9% █████████████████████████████
🟡 year                33,687 ✓   25,073 ✗   42.7% █████████████████████
🟡 title               40,830 ✓   17,930 ✗   30.5% ███████████████


In [13]:
# 🔍 ANALYSE: LÜCKEN BEI DATENSÄTZEN MIT ISBN/ISSN
print("\n🔍 === LÜCKEN BEI DATENSÄTZEN MIT ISBN/ISSN ===\n")

# Datensätze mit ISBN
has_isbn = df_vdeh['isbn'].notna()
isbn_count = has_isbn.sum()

if isbn_count > 0:
    print(f"📚 Datensätze mit ISBN: {isbn_count:,}\n")
    
    # Prüfe fehlende Felder bei ISBN-Datensätzen
    isbn_records = df_vdeh[has_isbn]
    
    missing_authors_isbn = (isbn_records['authors_str'].isna() | (isbn_records['authors_str'] == '')).sum()
    missing_publisher_isbn = isbn_records['publisher'].isna().sum()
    missing_year_isbn = isbn_records['year'].isna().sum()
    
    print(f"   Fehlende Autoren:  {missing_authors_isbn:6,} ({missing_authors_isbn/isbn_count*100:5.1f}%)")
    print(f"   Fehlender Verlag:  {missing_publisher_isbn:6,} ({missing_publisher_isbn/isbn_count*100:5.1f}%)")
    print(f"   Fehlendes Jahr:    {missing_year_isbn:6,} ({missing_year_isbn/isbn_count*100:5.1f}%)")
    
    # Mindestens ein Feld fehlt
    any_missing_isbn = (
        (isbn_records['authors_str'].isna() | (isbn_records['authors_str'] == '')) |
        isbn_records['publisher'].isna() |
        isbn_records['year'].isna()
    ).sum()
    
    print(f"\n   ⚠️  Mind. 1 Feld fehlt: {any_missing_isbn:6,} ({any_missing_isbn/isbn_count*100:5.1f}%)")
    print(f"   ✅ Alle Felder da:     {isbn_count - any_missing_isbn:6,} ({(isbn_count - any_missing_isbn)/isbn_count*100:5.1f}%)")

# Datensätze mit ISSN
has_issn = df_vdeh['issn'].notna()
issn_count = has_issn.sum()

if issn_count > 0:
    print(f"\n📰 Datensätze mit ISSN: {issn_count:,}\n")
    
    # Prüfe fehlende Felder bei ISSN-Datensätzen
    issn_records = df_vdeh[has_issn]
    
    missing_authors_issn = (issn_records['authors_str'].isna() | (issn_records['authors_str'] == '')).sum()
    missing_publisher_issn = issn_records['publisher'].isna().sum()
    missing_year_issn = issn_records['year'].isna().sum()
    
    print(f"   Fehlende Autoren:  {missing_authors_issn:6,} ({missing_authors_issn/issn_count*100:5.1f}%)")
    print(f"   Fehlender Verlag:  {missing_publisher_issn:6,} ({missing_publisher_issn/issn_count*100:5.1f}%)")
    print(f"   Fehlendes Jahr:    {missing_year_issn:6,} ({missing_year_issn/issn_count*100:5.1f}%)")
    
    # Mindestens ein Feld fehlt
    any_missing_issn = (
        (issn_records['authors_str'].isna() | (issn_records['authors_str'] == '')) |
        issn_records['publisher'].isna() |
        issn_records['year'].isna()
    ).sum()
    
    print(f"\n   ⚠️  Mind. 1 Feld fehlt: {any_missing_issn:6,} ({any_missing_issn/issn_count*100:5.1f}%)")
    print(f"   ✅ Alle Felder da:     {issn_count - any_missing_issn:6,} ({(issn_count - any_missing_issn)/issn_count*100:5.1f}%)")

# Datensätze mit ISBN ODER ISSN
has_isbn_or_issn = has_isbn | has_issn
isbn_or_issn_count = has_isbn_or_issn.sum()



🔍 === LÜCKEN BEI DATENSÄTZEN MIT ISBN/ISSN ===

📚 Datensätze mit ISBN: 11,415

   Fehlende Autoren:   3,423 ( 30.0%)
   Fehlender Verlag:   1,252 ( 11.0%)
   Fehlendes Jahr:       879 (  7.7%)

   ⚠️  Mind. 1 Feld fehlt:  3,734 ( 32.7%)
   ✅ Alle Felder da:      7,681 ( 67.3%)

📰 Datensätze mit ISSN: 721

   Fehlende Autoren:     667 ( 92.5%)
   Fehlender Verlag:      20 (  2.8%)
   Fehlendes Jahr:       650 ( 90.2%)

   ⚠️  Mind. 1 Feld fehlt:    680 ( 94.3%)
   ✅ Alle Felder da:         41 (  5.7%)
